<a href="https://colab.research.google.com/github/Pranav0-0Aggarwal/pygoat/blob/chatbot/chatbot/pygoatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install chatterbot==1.0.2


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 KB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 33.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.5/515.5 KB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.7/225.7 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.5/269.5 KB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.3/117.3 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.6/270.6 KB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sqlalchemy: filename=SQLAlchemy-1.2.19-cp39-cp39-linux_x86_64.whl size=1157816 sha256=922b29a3390946efcb35b47ccdfe5d923eeb8a2052229f7e2b11130708e7c628
  Stored in directory: /root/.cache/pip/wheels/12/75/ee/52355e3658b18c812a419a7fdf58

In [2]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
pip install logger

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for logger: filename=logger-1.4-py3-none-any.whl size=1780 sha256=375578a39f0783a695470050f7bfeed4759edc359a2f928ea57bbcf181fe7a17
  Stored in directory: /root/.cache/pip/wheels/d6/2c/20/0bf3428f60a053875adaf43bee20695fff6e5d5a2cffaceaad
Successfully built logger


In [35]:
training_data = [
    'What is OWASP PyGoat?',
    'OWASP PyGoat is an intentionally vulnerable web application used for learning web security testing.',
    'Why should I learn web security testing?',
    'Learning web security testing can help you understand how to identify and prevent web application attacks.',
    'What types of vulnerabilities can PyGoat help me learn about?',
    'PyGoat can help you learn about various types of web application vulnerabilities, including injection attacks, cross-site scripting (XSS), and broken authentication and session management.',
    'How can I use PyGoat to learn web security testing?',
    'PyGoat includes a series of lessons and challenges designed to teach you about web security testing techniques and common vulnerabilities.',
    'Is PyGoat suitable for beginners?',
    'Yes, PyGoat is designed to be accessible to beginners and experienced professionals alike.',
    'Where can I download PyGoat?',
    'You can download PyGoat from the official GitHub repository at https://github.com/OWASP/PyGoat',
    'Are there any resources available to help me get started with PyGoat?',
    'Yes, the PyGoat documentation includes a Getting Started guide and a list of additional resources to help you learn about web security testing.',
    'Can I contribute to PyGoat?',
    'Yes, PyGoat is an open-source project and welcomes contributions from anyone interested in improving the application.',
]


In [46]:
import nltk
import logging
from collections import Counter
from operator import itemgetter
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
from chatterbot.logic import BestMatch
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Download the required NLTK packages
nltk.download("punkt")
nltk.download("averaged_perceptron_tagger")
nltk.download("maxent_ne_chunker")
nltk.download("words")

# Define your training data here

# Create a new chatbot
logging.getLogger('chatterbot').setLevel(logging.ERROR)
chatbot = ChatBot(
    "PyGoatBot",
    storage_adapter="chatterbot.storage.SQLStorageAdapter",
    database_uri="sqlite:///database.sqlite3",
    logic_adapters=[
        {
            "import_path": "chatterbot.logic.BestMatch",
            "default_response": "I'm sorry, I'm not sure I understand.",
            "maximum_similarity_threshold": 0.90,
        }
    ],
)

# Train the chatbot with the training data
trainer = ListTrainer(chatbot)
trainer.train(training_data)
nltk.download('stopwords')

print("Welcome to PyGoatBot! Type 'q or exit' to quit.")
print()
print("How can I help?")
while True:
    try:
        user_input = input("You: ")
        if user_input.lower() == "exit" or user_input.lower()=="q":
            break
        sentence = user_input
        sentence = sentence.lower()
        words = word_tokenize(sentence)
        stop_words = set(stopwords.words('english'))
        words = [word for word in words if not word in stop_words]
        tagged_words = nltk.pos_tag(words)
        keywords = []
        for word, tag in tagged_words:
            if tag.startswith('N') or tag.startswith('J'):
                keywords.append(word)


        if not keywords:
            print("I'm sorry, I didn't find any important keywords in your input.")
            continue

        dict_que={}
        for i in range(0, len(training_data), 2):
            keyword_counts = 0
            for keyword in keywords:
                if keyword.lower() in training_data[i].lower():
                    keyword_counts += 1
            dict_que[training_data[i]]=keyword_counts

        sorted_dict = sorted(dict_que.items(), key=lambda x: x[1], reverse=True)
        top_questions=sorted_dict[0:4]

        if not top_questions:
            print("I'm sorry, I couldn't find any relevant questions in my database.")
            continue
        print()
        print("--Available questions:--")
        for i, question in enumerate(top_questions):
            print(f"   {i+1}. {question[0]}")
        print("   5. Main Menu")
        print()

        while True:
            try:
                question_index = int(input("Enter a number to select a question: "))
                if 1 <= question_index <= 4:
                    break
                elif question_index==5:
                    print("Taking you to the main menu")
                    break
                
                else:
                    print("Please enter a number between 1 and 4.")

            except ValueError:
                print("Please enter a valid number.")

        if question_index!=5:
          question = top_questions[question_index - 1][0]
          response = chatbot.get_response(question)

          # Output
          print(f"PyGoatBot: {response}")

    except (KeyboardInterrupt, EOFError):
        break


List Trainer: [####################] 100%

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloadi


Welcome to PyGoatBot! Type 'q or exit' to quit.
You: How to contribute in pygoat?

--Available questions:--
   1. Can I contribute to PyGoat?
   2. What is OWASP PyGoat?
   3. What types of vulnerabilities can PyGoat help me learn about?
   4. How can I use PyGoat to learn web security testing?
   5. Main Menu

Enter a number to select a question: 1
PyGoatBot: Yes, PyGoat is an open-source project and welcomes contributions from anyone interested in improving the application.
You: How to download pygoat?

--Available questions:--
   1. Where can I download PyGoat?
   2. What is OWASP PyGoat?
   3. What types of vulnerabilities can PyGoat help me learn about?
   4. How can I use PyGoat to learn web security testing?
   5. Main Menu

Enter a number to select a question: how to download?
Please enter a valid number.
Enter a number to select a question: 1
PyGoatBot: You can download PyGoat from the official GitHub repository at https://github.com/OWASP/PyGoat
You: what  is web security te